In [3]:
import pandas as pd
import geopandas as gpd
import folium

In [8]:
# Fetch the Minneapolis parks data
parks_url = "https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Parks/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"
parks_data = pd.read_json(parks_url)
# Convert the GeoJSON data to a DataFrame
parks_df = gpd.GeoDataFrame.from_features(parks_data["features"], crs="EPSG:4326")
parks_df.head()

,geometry,FID,OBJECTID,INT_REFNO,PREFIX,UDI,SYMBOL_NAM,REFNO,PARENT,SYMBOL_NUM,...,PARK_NAME1,PARK_NAME2,PARK_PARK3,GBS_PARK_A,PERIMETER,ACRES,REGIONAL,SHAPE_LENG,SHAPE_AREA,SHAPE_LEN
0,"POLYGON ((-93.30889 44.996, -93.30949 44.996, ...",1,28,-2150000000,REFNO,1195.27,PARKAREA,1195.27,PBSHADE,424,...,WILLARD,WILLARD,PARK,50443.117853,957.902574,1.158015,,955.815870,0,0
1,"POLYGON ((-93.31226 44.99, -93.31226 44.99, -9...",2,29,-2150000000,REFNO,1195.28,PARKAREA,1195.28,PBSHADE,424,...,FARWELL,FARWELL,PARK,50014.241634,882.384002,1.148169,,878.245473,0,0
2,"POLYGON ((-93.29092 44.93056, -93.29325 44.930...",3,31,-2150000000,REFNO,1195.35,PARKAREA,1195.35,PBSHADE,424,...,LYNDALE,LYNDALE,FARMSTEAD,750409.674557,3701.139805,17.227036,,4381.752389,0,0
3,"POLYGON ((-93.29991 44.95686, -93.29992 44.956...",4,32,-2150000000,REFNO,1195.37,PARKAREA,1195.37,PBSHADE,424,...,LEVIN,LEVIN,TRIANGLE,13239.957618,548.860887,0.303948,,566.882978,0,0
4,"POLYGON ((-93.29382 44.95972, -93.29407 44.959...",5,33,-2150000000,REFNO,1195.38,PARKAREA,1195.38,PBSHADE,424,...,SMITH,SMITH,TRIANGLE,11324.381447,510.308030,0.259972,,521.422418,0,0


In [ ]:
parks_df["PARK_PARK3"].value_counts()

PARK_PARK3
PARK              97
TRIANGLE          18
                  14
PARKWAY            9
CORRIDOR           8
FIELD              8
SQUARE             7
GREENWAY           3
TOTLOT             2
COURSE             2
PARK / PARKWAY     2
CIRCLE             1
MEADOWS            1
PLACE              1
FARMSTEAD          1
GARDENS            1
HEADQUARTERS       1
PARK / GOLF        1
HIGHWAY            1
GOLF / PARK        1
HILL               1
GOLF               1
LAKE               1
PARK / ROAD        1
OVAL               1
SITE               1
Name: count, dtype: int64

In [22]:
# Create a map centered on Minneapolis
m = folium.Map(location=[44.9778, -93.2650], zoom_start=12)
# Add the parks data to the map
popup = folium.GeoJsonPopup(
    fields=["PARK_NAME1", "PARK_PARK3"], aliases=["Park Name", "Park Type"]
)
tooltip = folium.GeoJsonTooltip(
    fields=["PARK_NAME1", "PARK_PARK3"], aliases=["Park Name", "Park Type"]
)
folium.GeoJson(
    parks_df,
    name="Minneapolis Parks",
    popup=popup,
    tooltip=tooltip,
    style_function=lambda x: {
        # Set the style for the parks based on PARK_PARK3 - PARK is green, TRIANGLE is orange, and other types are gray
        "fillColor": "green"
        if x["properties"]["PARK_PARK3"] == "PARK"
        else "orange"
        if x["properties"]["PARK_PARK3"] == "TRIANGLE"
        else "gray",
        "color": "darkgreen"
        if x["properties"]["PARK_PARK3"] == "PARK"
        else "darkorange"
        if x["properties"]["PARK_PARK3"] == "TRIANGLE"
        else "darkgray",
    },
).add_to(m)
folium.LayerControl().add_to(m)
# Display the map
m